
# ViEWS 3 constituent models 
## ViEWS production system, cm level


This notebook trains a set of regression models for use in the monthly updated ViEWS predicting fatalities ensemble

The notebook does the following: 
1. Retrieves data through querysets and stores in DataSets, a list of dictionaries
2. Specifies the metadata of a number of models, stores in ModelList, a list of dictionaries
3. Trains the models in ModelList, stores the trained objects in model storage and prediction storage
4. Saves part of ModelList as csv and the rest as pickles


## Importing modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import XGBRFRegressor, XGBRFClassifier

from lightgbm import LGBMClassifier, LGBMRegressor

# Views 3
from viewser.operations import fetch
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage
from views_runs.storage import store, retrieve, fetch_metadata

from views_forecasts.extensions import *

# Other packages
import pickle as pkl

# Packages from viewsforecasting repository

#from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated
import os
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
from FetchData import FetchData, RetrieveFromList, document_queryset, ReturnQsList, document_ensemble
from ViewsEstimators import *


## Common parameters

In [3]:
1+1

2

In [4]:
#!conda list | grep views

In [5]:
# To do:
# find out why and where missingness occurs

In [6]:
# Common parameters:
dev_id = 'Fatalities002'
run_id = dev_id

# Generating a new run if necessary

#try:
#    ViewsMetadata().new_run(name=run_id,description='Developing the fatalities model for FCDO',min_month=1,max_month=999)
#except KeyError:
#    if 'devel' not in run_id:
#        warnings.warn('You are overwriting a production system')

RerunQuerysets = True

FutureStart = 507
steps = [*range(1, 36+1, 1)] # Which steps to train and predict for
fi_steps = [1,3,6,12,36] # Which steps to present feature importances for
#steps = [1,3,6,12,36]
#fi_steps = [1,3,6,12,36]

# Specifying partitions
calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{os.getlogin()}/Dropbox (ViEWS)/ViEWS'
print('Setting Mydropbox to',Mydropbox)

Setting Mydropbox to /Users/jim/Dropbox (ViEWS)/ViEWS


# Retrieve data

In [7]:
# Create Markdown documentation of all querysets used
level = 'cm'
qslist = ReturnQsList(level)
document_queryset(qslist,dev_id)

 .    fatalities002_baseline; A dataset with 6 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_topics_stub; A dataset with 62 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_aquastat_stub; A dataset with 11 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_cm_conflict_history_stub; A dataset with 24 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_cm_conflict_history_ext; A dataset with 33 columns, with data between t = 1 and 852. (213 units)
 .    fatalities002_vdem_short_stub; A dataset with 58 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_wdi_short_stub; A dataset with 28 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_joint_narrow; A dataset with 39 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_joint_broad_stub; A dataset with 102 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_faostat_stub;A

In [8]:
from FetchData import fetch_cm_data_from_model_def

Datasets=fetch_cm_data_from_model_def(qslist)

 .    faoprices: A dataset with 17 columns, with data between t = 1 and 852; 213 units.
 .    faostat: A dataset with 41 columns, with data between t = 1 and 852; 213 units.
 .    topics_002: A dataset with 68 columns, with data between t = 1 and 852; 213 units.
 .    vdem_short: A dataset with 64 columns, with data between t = 1 and 852; 213 units.
 .    all_features: A dataset with 189 columns, with data between t = 1 and 852; 213 units.
 .    joint_broad: A dataset with 108 columns, with data between t = 1 and 852; 213 units.
 .    aquastat: A dataset with 17 columns, with data between t = 1 and 852; 213 units.
 .    imfweo: A dataset with 11 columns, with data between t = 1 and 852; 213 units.
 .    joint_narrow: A dataset with 39 columns, with data between t = 1 and 852; 213 units.
 .    baseline002: A dataset with 6 columns, with data between t = 1 and 852; 213 units.
 .    conflictlong_ln: A dataset with 63 columns, with data between t = 1 and 852; 213 units.
 .    conflict_ln: 

In [57]:
Datasets[4]['df']['vdem_v2clrgunev'].head()

month_id  country_id
1         1             0.697
          2             1.151
          3             0.615
          4            -0.687
          5               NaN
Name: vdem_v2clrgunev, dtype: float64

# Generating predictions
Using the ViEWS3 partitioning/stepshifting syntax. Training models for A: calibration partition and B: test partition, to test out some calibration routines. Most models trained with ln_ged_sb_best as outcome.

In [56]:
Datasets[0]['df']

fao_wheat_price  fao_mp_price  fao_sugar_price  \
month_id country_id                                                   
1        1                       0.0           0.0              0.0   
         2                       0.0           0.0              0.0   
         3                       0.0           0.0              0.0   
         4                       0.0           0.0              0.0   
         5                       0.0           0.0              0.0   
...                              ...           ...              ...   
852      242                     0.0           0.0              0.0   
         243                     0.0           0.0              0.0   
         244                     0.0           0.0              0.0   
         245                     0.0           0.0              0.0   
         246                     0.0           0.0              0.0   

                     fao_meat_price  fao_milk_price  ln_ged_sb_dep  ln_ged_sb  \
month_id country_id                                                             
1        1                      0.0             0.0       0.000000   0.000000   
         2                      0.0             0.0       0.000000   0.000000   
         3                      0.0             0.0       0.000000   0.000000   
         4                      0.0             0.0       0.000000   0.000000   
         5                      0.0             0.0       0.000000   0.000000   
...                             ...             ...            ...        ...   
852      242                    0.0             0.0       0.000000   0.000000   
         243                    0.0             0.0       0.000000   0.000000   
         244                    0.0             0.0       0.000000   0.000000   
         245                    0.0             0.0       2.079442   2.079442   
         246                    0.0             0.0       1.609438   1.609438   

                     gleditsch_ward  delta_fao_wheat_price  \
month_id country_id                                          
1        1                    110.0                    0.0   
         2                    115.0                    0.0   
         3                     52.0                    0.0   
         4                    101.0                    0.0   
         5                    990.0                    0.0   
...                             ...                    ...   
852      242                  510.0                    0.0   
         243                  600.0                    0.0   
         244                  435.0                    0.0   
         245                  625.0                    0.0   
         246                  626.0                    0.0   

                     delta_fao_mp_price  delta_fao_sugar_price  \
month_id country_id                                              
1        1                          0.0                    0.0   
         2                          0.0                    0.0   
         3                          0.0                    0.0   
         4                          0.0                    0.0   
         5                          0.0                    0.0   
...                                 ...                    ...   
852      242                        0.0                    0.0   
         243                        0.0                    0.0   
         244                        0.0                    0.0   
         245                        0.0                    0.0   
         246                        0.0                    0.0   

                     delta_fao_meat_price  delta_fao_milk_price  \
month_id country_id                                               
1        1                            0.0                   0.0   
         2                            0.0                   0.0   
         3                            0.0                   0.0   
         4                            0.0 

In [55]:
dev_id

'Fatalities002'

In [11]:
Datasets[0]['df'].head()

fao_wheat_price  fao_mp_price  fao_sugar_price  \
month_id country_id                                                   
1        1                       0.0           0.0              0.0   
         2                       0.0           0.0              0.0   
         3                       0.0           0.0              0.0   
         4                       0.0           0.0              0.0   
         5                       0.0           0.0              0.0   

                     fao_meat_price  fao_milk_price  ln_ged_sb_dep  ln_ged_sb  \
month_id country_id                                                             
1        1                      0.0             0.0            0.0        0.0   
         2                      0.0             0.0            0.0        0.0   
         3                      0.0             0.0            0.0        0.0   
         4                      0.0             0.0            0.0        0.0   
         5                      0.0             0.0            0.0        0.0   

                     gleditsch_ward  delta_fao_wheat_price  \
month_id country_id                                          
1        1                    110.0                    0.0   
         2                    115.0                    0.0   
         3                     52.0                    0.0   
         4                    101.0                    0.0   
         5                    990.0                    0.0   

                     delta_fao_mp_price  delta_fao_sugar_price  \
month_id country_id                                              
1        1                          0.0                    0.0   
         2                          0.0                    0.0   
         3                          0.0                    0.0   
         4                          0.0                    0.0   
         5                          0.0                    0.0   

                     delta_fao_meat_price  delta_fao_milk_price  \
month_id country_id                                               
1        1                            0.0                   0.0   
         2                            0.0                   0.0   
         3                            0.0                   0.0   
         4                            0.0                   0.0   
         5                            0.0                   0.0   

                     wdi_sp_pop_totl  decay_ged_sb_5  decay_ged_os_5  \
month_id country_id                                                    
1        1                  780153.0             0.0             0.0   
         2                  359531.0             0.0             0.0   
         3                 1084744.0             0.0             0.0   
         4                15182611.0             0.0             0.0   
         5                  155525.0             0.0             0.0   

                     splag_1_decay_ged_sb_5  
month_id country_id                          
1        1                              0.0  
         2                              0.0  
         3                              0.0  
         4                              0.0  
         5                              0.0

In [ ]:
from views_runs import ModelMetadata 
help(ModelMetadata)

## Checking missingness and infinity values

In [12]:
N=51
for i in range(len(Datasets)):
    df = Datasets[i]['df']
    print(Datasets[i]['Name'])
    for col in df.iloc[: , :N].columns:
        print(col,len(df[col]), 'missing:', df[col].isnull().sum(), 'infinity:', np.isinf(df).values.sum())


faoprices
fao_wheat_price 158230 missing: 0 infinity: 0
fao_mp_price 158230 missing: 0 infinity: 0
fao_sugar_price 158230 missing: 0 infinity: 0
fao_meat_price 158230 missing: 0 infinity: 0
fao_milk_price 158230 missing: 0 infinity: 0
ln_ged_sb_dep 158230 missing: 0 infinity: 0
ln_ged_sb 158230 missing: 0 infinity: 0
gleditsch_ward 158230 missing: 0 infinity: 0
delta_fao_wheat_price 158230 missing: 0 infinity: 0
delta_fao_mp_price 158230 missing: 0 infinity: 0
delta_fao_sugar_price 158230 missing: 0 infinity: 0
delta_fao_meat_price 158230 missing: 0 infinity: 0
delta_fao_milk_price 158230 missing: 0 infinity: 0
wdi_sp_pop_totl 158230 missing: 11 infinity: 0
decay_ged_sb_5 158230 missing: 0 infinity: 0
decay_ged_os_5 158230 missing: 0 infinity: 0
splag_1_decay_ged_sb_5 158230 missing: 0 infinity: 0
faostat
ln_ged_sb_dep 158230 missing: 0 infinity: 0
ln_ged_sb 158230 missing: 0 infinity: 0
gleditsch_ward 158230 missing: 0 infinity: 0
wdi_sp_pop_totl 158230 missing: 11 infinity: 0
consume

ln_ged_sb 158230 missing: 0 infinity: 0
ln_ged_ns 158230 missing: 0 infinity: 0
ln_ged_os 158230 missing: 0 infinity: 0
ln_acled_sb 158230 missing: 1437 infinity: 0
ln_acled_sb_count 158230 missing: 1437 infinity: 0
ln_acled_os 158230 missing: 1437 infinity: 0
wdi_sp_pop_totl 158230 missing: 11 infinity: 0
wdi_ag_lnd_frst_k2 158230 missing: 11 infinity: 0
wdi_dt_oda_odat_pc_zs 158230 missing: 11 infinity: 0
wdi_ms_mil_xpnd_gd_zs 158230 missing: 11 infinity: 0
wdi_ms_mil_xpnd_zs 158230 missing: 11 infinity: 0
wdi_nv_agr_totl_kd 158230 missing: 11 infinity: 0
wdi_nv_agr_totl_kn 158230 missing: 11 infinity: 0
wdi_ny_gdp_pcap_kd 158230 missing: 11 infinity: 0
wdi_sp_dyn_le00_in 158230 missing: 11 infinity: 0
wdi_se_enr_prim_fm_zs 158230 missing: 11 infinity: 0
wdi_se_enr_prsc_fm_zs 158230 missing: 11 infinity: 0
wdi_se_prm_nenr 158230 missing: 11 infinity: 0
wdi_sh_sta_maln_zs 158230 missing: 11 infinity: 0
wdi_sh_sta_stnt_zs 158230 missing: 11 infinity: 0
wdi_sl_tlf_totl_fe_zs 158230 miss

ste_theta11_stock 158230 missing: 5 infinity: 0
ste_theta13_stock 158230 missing: 5 infinity: 0
ste_theta14_stock 158230 missing: 5 infinity: 0
splag_ste_theta2_stock 158230 missing: 0 infinity: 0
splag_ste_theta11_stock 158230 missing: 0 infinity: 0
splag_ste_theta13_stock 158230 missing: 0 infinity: 0
baseline002
ln_ged_sb_dep 158230 missing: 0 infinity: 0
ln_ged_sb 158230 missing: 0 infinity: 0
wdi_sp_pop_totl 158230 missing: 11 infinity: 0
decay_ged_sb_5 158230 missing: 0 infinity: 0
decay_ged_os_5 158230 missing: 0 infinity: 0
splag_1_decay_ged_sb_5 158230 missing: 0 infinity: 0
conflictlong_ln
gleditsch_ward 158230 missing: 0 infinity: 0
ln_ged_sb_dep 158230 missing: 0 infinity: 0
ln_ged_sb 158230 missing: 0 infinity: 0
ln_ged_ns 158230 missing: 0 infinity: 0
ln_ged_os 158230 missing: 0 infinity: 0
ln_acled_sb 158230 missing: 1437 infinity: 0
ln_acled_sb_count 158230 missing: 1437 infinity: 0
ln_acled_os 158230 missing: 1437 infinity: 0
splag_1_ged_sb 158230 missing: 0 infinity: 

pc7 74316 missing: 0 infinity: 0
pc8 74316 missing: 0 infinity: 0
pc9 74316 missing: 0 infinity: 0
pc10 74316 missing: 0 infinity: 0
pc11 74316 missing: 0 infinity: 0
pc12 74316 missing: 0 infinity: 0
pc13 74316 missing: 0 infinity: 0
pc14 74316 missing: 0 infinity: 0
pc15 74316 missing: 0 infinity: 0
pca_wdi
ln_ged_sb_dep 74316 missing: 0 infinity: 0
ln_ged_sb 74316 missing: 0 infinity: 0
decay_ged_sb_5 74316 missing: 0 infinity: 0
decay_ged_os_5 74316 missing: 0 infinity: 0
splag_1_decay_ged_sb_5 74316 missing: 0 infinity: 0
wdi_sp_pop_totl 74316 missing: 11 infinity: 0
pc1 74316 missing: 0 infinity: 0
pc2 74316 missing: 0 infinity: 0
pc3 74316 missing: 0 infinity: 0
pc4 74316 missing: 0 infinity: 0
pc5 74316 missing: 0 infinity: 0
pc6 74316 missing: 0 infinity: 0
pc7 74316 missing: 0 infinity: 0
pc8 74316 missing: 0 infinity: 0
pc9 74316 missing: 0 infinity: 0
pc10 74316 missing: 0 infinity: 0
pc11 74316 missing: 0 infinity: 0
pc12 74316 missing: 0 infinity: 0
pc13 74316 missing: 0 

# Specify models in ensemble

In [13]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels('cm')
    
for imodel,model in enumerate(ModelList):
    print(imodel, model['modelname'], model['data_train'])

0 fatalities002_baseline_rf baseline002
1 fatalities002_conflicthistory_rf conflict_ln
2 fatalities002_conflicthistory_gbm conflict_ln
3 fatalities002_conflicthistory_hurdle_lgb conflict_ln
4 fatalities002_conflicthistory_long_xgb conflictlong_ln
5 fatalities002_vdem_hurdle_xgb vdem_short
6 fatalities002_wdi_rf wdi_short
7 fatalities002_topics_rf topics_002
8 fatalities002_topics_xgb topics_002
9 fatalities002_topics_hurdle_lgb topics_002
10 fatalities002_joint_broad_rf joint_broad
11 fatalities002_joint_broad_hurdle_rf joint_broad
12 fatalities002_joint_narrow_xgb joint_narrow
13 fatalities002_joint_narrow_hurdle_xgb joint_narrow
14 fatalities002_joint_narrow_hurdle_lgb joint_narrow
15 fatalities002_all_pca3_xgb all_features
16 fatalities002_aquastat_rf aquastat
17 fatalities002_faostat_rf faostat
18 fatalities002_faoprices_rf faoprices
19 fatalities002_imfweo_rf imfweo
20 fat_hh20_Markov_glm joint_narrow
21 fat_hh20_Markov_rf joint_narrow


In [14]:
ModelList

[{'modelname': 'fatalities002_baseline_rf',
  'algorithm': XGBRFRegressor(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bytree=None,
                 early_stopping_rounds=None, enable_categorical=False,
                 eval_metric=None, gamma=None, gpu_id=None, grow_policy=None,
                 importance_type=None, interaction_constraints=None, max_bin=None,
                 max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
                 max_leaves=None, min_child_weight=None, missing=nan,
                 monotone_constraints=None, n_estimators=300, n_jobs=12,
                 num_parallel_tree=None, objective='reg:squarederror',
                 predictor=None, random_state=None, reg_alpha=None,
                 sampling_method=None, scale_pos_weight=None, ...),
  'depvar': 'ln_ged_sb_dep',
  'data_train': 'baseline002',
  'queryset': 'fatalities002_baseline',
  'preprocessing': 'float_it',
  'level': 'cm',
  'desc

In [15]:
document_ensemble(ModelList,'sb')

0 fatalities002_baseline_rf baseline002
1 fatalities002_conflicthistory_rf conflict_ln
2 fatalities002_conflicthistory_gbm conflict_ln
3 fatalities002_conflicthistory_hurdle_lgb conflict_ln
4 fatalities002_conflicthistory_long_xgb conflictlong_ln
5 fatalities002_vdem_hurdle_xgb vdem_short
6 fatalities002_wdi_rf wdi_short
7 fatalities002_topics_rf topics_002
8 fatalities002_topics_xgb topics_002
9 fatalities002_topics_hurdle_lgb topics_002
10 fatalities002_joint_broad_rf joint_broad
11 fatalities002_joint_broad_hurdle_rf joint_broad
12 fatalities002_joint_narrow_xgb joint_narrow
13 fatalities002_joint_narrow_hurdle_xgb joint_narrow
14 fatalities002_joint_narrow_hurdle_lgb joint_narrow
15 fatalities002_all_pca3_xgb all_features
16 fatalities002_aquastat_rf aquastat
17 fatalities002_faostat_rf faostat
18 fatalities002_faoprices_rf faoprices
19 fatalities002_imfweo_rf imfweo
20 fat_hh20_Markov_glm joint_narrow
21 fat_hh20_Markov_rf joint_narrow


In [48]:
# Loop that checks whether the model exists, retrains if not, 
# and stores the predictions if they have not been stored before for this run.
# To do: set the data_preprocessing to the function in the model dictionary

level = 'cm'
includeFuture = False

from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from viewser import Queryset, Column
from views_runs import operations
from views_runs.run_result import RunResult

i = 0
for model in ModelList[5:6]:
    if model['algorithm'] != 'Rscript':
        force_retrain = True
        modelstore = storage.Storage()
        ct = datetime.now()
        print(i, model['modelname'])
        print('Calibration partition', ct)
        model['Algorithm_text'] = str(model['algorithm'])
        model['RunResult_calib'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"calib":calib_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "calib",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_calib',
                author_name        = "HH",
        )

    #    model['predstore_calib'] = level +  '_' + model['modelname'] + '_calib'
        ct = datetime.now()
        print('Trying to retrieve predictions', ct)
        try:
            predictions_calib = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_calib'])
        except KeyError:
            print(model['predstore_calib'], ', run',  run_id, 'does not exist, predicting')
            predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
            predictions_calib.forecasts.set_run(run_id)
            predictions_calib.forecasts.to_store(name=model['predstore_calib'])

        ct = datetime.now()
        print('Test partition', ct)
        modelstore = storage.Storage()
        model['RunResult_test'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"test":test_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "test",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_test',
                author_name        = "HH",
        )
        ct = datetime.now()
        print('Trying to retrieve predictions', ct)
    #    model['predstore_test'] = level +  '_' + model['modelname'] + '_test'
        try:
            predictions_test = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_test'])
        except KeyError:
            print(model['predstore_test'], ', run', run_id, 'does not exist, predicting')
            predictions_test = model['RunResult_test'].run.predict("test","predict",model['RunResult_test'].data)
            predictions_test.forecasts.set_run(run_id)
            predictions_test.forecasts.to_store(name=model['predstore_test'])
        # Predictions for true future
        if includeFuture:
            ct = datetime.now()
            print('Future', ct)
            modelstore = storage.Storage()
            model['RunResult_future'] = RunResult.retrain_or_retrieve(
                    retrain            = force_retrain,
                    store              = modelstore,
                    partitioner        = DataPartitioner({"test":future_partitioner_dict}),
                    stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                    dataset            = RetrieveFromList(Datasets,model['data_train']),
                    queryset_name      = model['queryset'],
                    partition_name     = "test",
                    timespan_name      = "train",
                    storage_name       = model['modelname'] + '_future',
                    author_name        = "HH",
            )
            ct = datetime.now()
            print('Trying to retrieve predictions', ct)
            model['predstore_future'] = level +  '_' + model['modelname'] + '_f' + str(FutureStart)
            try:
                predictions_future = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_future'])
            except KeyError:
                print(model['predstore_future'], ', run', run_id, 'does not exist, predicting')
                predictions_future = model['RunResult_future'].run.future_point_predict(FutureStart,model['RunResult_future'].data)
                predictions_future.forecasts.set_run(run_id)
                predictions_future.forecasts.to_store(name=model['predstore_future'])  
        print('**************************************************************')
    i = i + 1

print('All done')

0 fatalities002_vdem_hurdle_xgb
Calibration partition 2022-10-25 10:20:40.941969
 * == Performing a run: "fatalities002_vdem_hurdle_xgb_calib" == * 
Model object named "fatalities002_vdem_hurdle_xgb_calib" with equivalent metadata already exists.
Retrain is true, overwriting "fatalities002_vdem_hurdle_xgb_calib"
Training model(s)...
Storing "fatalities002_vdem_hurdle_xgb_calib"
Trying to retrieve predictions 2022-10-25 10:22:55.900610
pr_46_cm_fatalities002_vdem_hurdle_xgb_calib.parquet
Test partition 2022-10-25 10:22:58.515302
 * == Performing a run: "fatalities002_vdem_hurdle_xgb_test" == * 
Model object named "fatalities002_vdem_hurdle_xgb_test" with equivalent metadata already exists.
Retrain is true, overwriting "fatalities002_vdem_hurdle_xgb_test"
Training model(s)...
Storing "fatalities002_vdem_hurdle_xgb_test"
Trying to retrieve predictions 2022-10-25 10:25:47.707109
pr_46_cm_fatalities002_vdem_hurdle_xgb_test.parquet
************************************************************

In [22]:
# Exploring the future predictions


predictions_test.xs(246,level=1).tail()

,imfweo_ngdp_rpch_tcurrent,imfweo_ngdp_rpch_tmin1,imfweo_ngdp_rpch_tplus1,imfweo_ngdp_rpch_tplus2,ln_ged_sb_dep,gleditsch_ward,ln_ged_sb,wdi_sp_pop_totl,decay_ged_sb_5,decay_ged_os_5,...,step_pred_33,step_pred_34,step_pred_35,step_pred_36,step_pred_4,step_pred_5,step_pred_6,step_pred_7,step_pred_8,step_pred_9
month_id,,,,,,,,,,,,,,,,,,,,,
488,0.0,0.0,0.0,0.0,3.218876,626.0,3.218876,11062113.0,1.000000,1.000000,...,3.913047,3.388252,3.466939,3.391691,1.709457,1.647634,1.139124,1.967838,2.335133,1.886065
489,0.0,0.0,0.0,0.0,0.693147,626.0,0.693147,11062113.0,1.000000,0.971532,...,3.839852,3.781158,3.137412,3.756255,2.613080,1.548834,2.048799,1.024652,2.154551,1.861133
490,0.0,0.0,0.0,0.0,1.609438,626.0,1.609438,11062113.0,0.971532,0.943874,...,3.369635,3.725624,3.738224,3.355103,2.046697,2.755813,1.464292,1.738210,1.514066,1.852678
491,0.0,0.0,0.0,0.0,2.484907,626.0,2.484907,11062113.0,0.943874,0.917004,...,3.637645,3.567117,3.625956,3.974339,2.455626,1.925900,2.770793,1.607153,2.113373,1.534434
492,0.0,0.0,0.0,0.0,0.000000,626.0,0.000000,11062113.0,1.000000,0.890899,...,3.243661,3.710055,3.410763,3.963485,2.856974,2.578416,2.112898,2.429112,1.426705,1.302743


## Notes on training time for the various algorithms:

In [ ]:
#These are calculated in minutes for the hh20 feature set (with about 40 features), for all 36 steps, calibration (c) and test (t) partitions, also include generating predictions, and are approximate:

#nj=12 (number of threads)
#scikit random forest:        21:13 (c), 26:20 (t) RandomForestRegressor(n_estimators=200, n_jobs=nj)
#XGB random forest:           06:02 (c), 07:51 (t) XGBRFRegressor(n_estimators=300,n_jobs=nj)
#scikit gbm:                  13:59 (c), 15:55 (t) GradientBoostingRegressor(), 
#scikit hurdle random forest: 07:32 (c), 09:49 (t) For both clf and reg: (n_estimators=200, n_jobs=nj)
#XGB hurdle xgb:              01:26 (c), 01:32 (t) For both clf and reg:                n_estimators=200,tree_method='hist',n_jobs=nj)
#scikit histgbm:              01:17 (c), 01:20 (t) HistGradientBoostingRegressor(max_iter=200)
#XGB xgb:                     01:00 (c), 01:04 (t) XGBRegressor(n_estimators=200,tree_method='hist',n_jobs=nj)
#lightgbm gbm:                00:25 (c), --    (t) LGBMRegressor(n_estimators=100,num_threads=8)

# Various helper functions and tools....

In [ ]:
!conda list | grep views-forecasts

# Retrieving external forecasts

In [23]:
# Retrieve David's Markov models
# To do: rewrite the model dictionary to the new, slimmer version.
DRList = []


model = {
    'modelname':   'fat_hh20_Markov_glm',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}
DRList.append(model)

model = {
    'modelname':   'fat_hh20_Markov_rf',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}

DRList.append(model)



In [40]:
path = f'/Users/{os.getlogin()}/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/Predictions/cm/preds/'

DRList[0]['predictions_file_calib'] = path + 'vmm_glm_hh20_0125_alt_calib.csv'
DRList[0]['predictions_file_test'] = path + 'vmm_glm_hh20_0125_alt_test.csv'
DRList[0]['predictions_file_future'] = path + 'vmm_glm_hh20_506.csv'

DRList[1]['predictions_file_calib'] = path + 'vmm_rf_hh20_0125_alt_calib.csv'
DRList[1]['predictions_file_test'] = path + 'vmm_rf_hh20_0125_alt_test.csv'
DRList[1]['predictions_file_future'] = path + 'vmm_rf_hh20_505.csv'

In [41]:
print(path)

/Users/jim/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/Predictions/cm/preds/


In [42]:

for model in ModelList:
    print(model['modelname'])

fatalities002_baseline_rf
fatalities002_conflicthistory_rf
fatalities002_conflicthistory_gbm
fatalities002_conflicthistory_hurdle_lgb
fatalities002_conflicthistory_long_xgb
fatalities002_vdem_hurdle_xgb
fatalities002_wdi_rf
fatalities002_topics_rf
fatalities002_topics_xgb
fatalities002_topics_hurdle_lgb
fatalities002_joint_broad_rf
fatalities002_joint_broad_hurdle_rf
fatalities002_joint_narrow_xgb
fatalities002_joint_narrow_hurdle_xgb
fatalities002_joint_narrow_hurdle_lgb
fatalities002_all_pca3_xgb
fatalities002_aquastat_rf
fatalities002_faostat_rf
fatalities002_faoprices_rf
fatalities002_imfweo_rf
fat_hh20_Markov_glm
fat_hh20_Markov_rf


In [45]:
# Storing Markov models in central storage
# Retrieving dependent variable
target_calib = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_calib', run=run_id)['ln_ged_sb_dep']
target_test = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_test', run=run_id)['ln_ged_sb_dep']
level = 'cm'
for model in DRList:
    df_calib = pd.read_csv(model['predictions_file_calib'],index_col=['month_id','country_id'])
    df_test = pd.read_csv(model['predictions_file_test'],index_col=['month_id','country_id'])
    df_future = pd.read_csv(model['predictions_file_future'],index_col=['month_id','country_id'])
    df_calib['ln_ged_sb_dep'] = target_calib
    df_test['ln_ged_sb_dep'] = target_test
    df_future['ln_ged_sb_dep'] = np.nan # Empty dependent variable column for consistency/required by prediction storage function
    stored_modelname = level + '_' + model['modelname'] + '_calib'
    df_calib.forecasts.set_run(run_id)
    df_calib.forecasts.to_store(name=stored_modelname, overwrite=True)
    stored_modelname = level + '_' + model['modelname'] + '_test'
    df_test.forecasts.set_run(run_id)
    df_test.forecasts.to_store(name=stored_modelname, overwrite=True)    

pr_46_cm_fat_conflicthistory_rf_calib.parquet
pr_46_cm_fat_conflicthistory_rf_test.parquet


In [46]:
ModelList

[{'modelname': 'fatalities002_baseline_rf',
  'algorithm': XGBRFRegressor(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bytree=None,
                 early_stopping_rounds=None, enable_categorical=False,
                 eval_metric=None, gamma=None, gpu_id=None, grow_policy=None,
                 importance_type=None, interaction_constraints=None, max_bin=None,
                 max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
                 max_leaves=None, min_child_weight=None, missing=nan,
                 monotone_constraints=None, n_estimators=300, n_jobs=12,
                 num_parallel_tree=None, objective='reg:squarederror',
                 predictor=None, random_state=None, reg_alpha=None,
                 sampling_method=None, scale_pos_weight=None, ...),
  'depvar': 'ln_ged_sb_dep',
  'data_train': 'baseline002',
  'queryset': 'fatalities002_baseline',
  'preprocessing': 'float_it',
  'level': 'cm',
  'desc